In [1]:
import pickle

import gensim
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import twint
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier


In [4]:
import os

dir = './emotions_prediction'
print(os.getcwd())

with open("{}/data/dataset.raw.pickle".format(dir), "rb") as dataset_file:
    dataset = pickle.load(dataset_file, encoding='latin1')


/Users/r.makowiecki/PycharmProjects/SocialMediaAnalysis


In [5]:
labels = ['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']

# print dataset type and sample
print(type(dataset))
print(dataset['info'][:10])
print(dataset['texts'][:10])

<class 'dict'>
[{'label': array([1., 0., 0., 0., 0., 0., 0.])}, {'label': array([0., 1., 0., 0., 0., 0., 0.])}, {'label': array([0., 0., 1., 0., 0., 0., 0.])}, {'label': array([0., 0., 0., 1., 0., 0., 0.])}, {'label': array([0., 0., 0., 0., 1., 0., 0.])}, {'label': array([0., 0., 0., 0., 0., 1., 0.])}, {'label': array([0., 0., 0., 0., 0., 0., 1.])}, {'label': array([1., 0., 0., 0., 0., 0., 0.])}, {'label': array([0., 1., 0., 0., 0., 0., 0.])}, {'label': array([0., 0., 1., 0., 0., 0., 0.])}]
['During the period of falling in love, each time that we met and especially when we had not met for a long time.', 'When I was involved in a traffic accident.', 'When I was driving home after  several days of hard work, there was a motorist ahead of me who was driving at 50 km/hour and refused, despite his low speeed to let me overtake.', 'When I lost the person who meant the most to me.', "The time I knocked a deer down - the sight of the animal's injuries and helplessness.  The realization that t

In [6]:
X = dataset['texts']
Y = [item['label'] for item in dataset['info']]
    
#print labels sample
print(Y[:10])

[array([1., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0.]), array([0., 0., 0., 0., 0., 0., 1.]), array([1., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0.])]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [8]:
# model taken from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM. Place in appropriate directory
model = KeyedVectors.load_word2vec_format('./emotions_prediction/model/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [9]:
def get_english_stopwords():
    file_path = './emotions_prediction/data/nltk_english_stopwords'
    with open(file_path, 'r') as stop_words_file:
        return stop_words_file.read().splitlines()


In [23]:

stop_words_list = set('for a of the and to in'.split())

def vectorize_sentence(model_w2v, sentence, stop_words=stop_words_list):
    words = gensim.utils.simple_preprocess(sentence)

    sentence_vector = []
    for word in words:
        if word not in stop_words and word in model_w2v.vocab:
            sentence_vector.append(model_w2v[word])

    sentence_vector = np.stack(sentence_vector, axis=0)

    return np.mean(sentence_vector, axis=0)

In [25]:
print(X_train[:10])
X_train_vectorized = [vectorize_sentence(model, sentence) for sentence in X_train]
X_test_vectorized = [vectorize_sentence(model, sentence) for sentence in X_test]

y_test = np.stack(y_test, axis=0)
print(y_test)


['When my aunt took her own life, I felt guity for not understanding that she needed support.', 'When I was house-hunting: Another person got a flat that I would have liked, too, because he heaped a floot of words on the letter.', 'When I was 4-5 years old my mother gave me some parts of a chocolate bar and ordered me to bring it to my father working outside. On my way I could not resist the temptation and ate it myself.', 'I passed an exam which I thought I had failed.', 'A friend of mine told me that my boyfriend had been with another girl this Summer.  We had been going out together for four years and we had not done so for the last three months.  I could not believe it.', 'My parents put pressure on me to buy clothes, when I would not have liked to do so.', 'When my brothers had passed all of their exams and were able to  graduate from their courses.', 'I read a book about the sexual phantasies of women; I read about a woman having sexual intercourse with a dog.', 'I heard that a f

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
